<a href="https://colab.research.google.com/github/abhishek085/Transformers_LM_applications/blob/main/Transformers_prompt_engineering_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Summarization of Dialogue using Transformers

In [4]:
!pip install --upgrade pip
!pip install --disbale-pip-version-check \ torch==1.13.1 \torchdata==0.5.1 --quiet

!pip install \ transformers==4.27.2 \ datasets==2.11.0 --quiet


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --disbale-pip-version-check
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.2 MB/s eta 0:00:00


Load the datasets, LM,tokenizer and configurator

In [5]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

Summarizing the dialogues data set without any Prompt Engineering input (RAW attempt at how the model works)

In [6]:
huggingface_dataset_name = 'knkarthick/dialogsum'
dataset = load_dataset(huggingface_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Lets check the dataset

In [8]:
example_indices = [40,200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
  print(dash_line)
  print('Example',i+1)
  print(dash_line)
  print('INPUT DIALOGUE:')
  print(dataset['test'][index]['dialogue'])
  print(dash_line)



---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------
Example 2
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure wha

Checking Human summaries of these dialogues

In [9]:
example_indices = [40,200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
  print(dash_line)
  print('Example',i+1)
  print(dash_line)
  print('INPUT DIALOGUE:')
  print(dataset['test'][index]['dialogue'])
  print(dash_line)
  print('HUMAN SUMMARY:')
  print(dataset['test'][index]['summary'])
  print(dash_line)

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------
Example 2
----

load the model FLAN-T5 model, create an instance of AutoModelforSeq2SeqLM class from pretrained method of transformers library

In [10]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

To perform Encoding and decoding of transformer architecture we need to convert text to numbers using Tokenizer ( tokenization splits the text into smaller unites whcih can be processed by the LLM Models)

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast = True)
#use fast switch on the fast tokenizer

Testing how tokenizer is actually working on a raw random text

In [14]:
sentence = "My name is Tom and my train is late"
sentence_encoded = tokenizer(sentence,return_tensors = 'pt')

sentence_decoded = tokenizer.decode(
    sentence_encoded["input_ids"][0],
    skip_special_tokens = True
)

print('Endoed Senetence:')
print(sentence_encoded["input_ids"])
print('\n Decoded sentence :')
print(sentence_decoded)


Endoed Senetence:
tensor([[ 499,  564,   19, 3059,   11,   82, 2412,   19, 1480,    1]])

 Decoded sentence :
My name is Tom and my train is late


# Checking how the model is working with out any prompt give and asked to summarize directly -- WITHOUT GIVING ANY PROMPT (PROMPT ENGINEERING )

In [15]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  inputs = tokenizer(dialogue,return_tensors ='pt')
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],max_new_tokens = 50,)[0],
      skip_special_tokens = True

      )

  print(dash_line)
  print('Example',i+1)
  print(dash_line)
  print(f'INPUT PROMPT: \n{dialogue}')
  print(dash_line)
  print(f'Human Summary: \n{summary}')
  print(dash_line)
  print(f'Model Summary:\n{output}')



---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Human Summary: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Summary:
Person1: It's ten to nine.
------------------------------------------------------------------------

NOW LETS SEE IF WE GIVE A PROMPT THEN how the situation changes or affects this model (LM)

ZERO - SHOT INFERENCE WITH AN INSTRUCTION PROMPT ( This is basically giving some prompt but not telling the model how the sample answer looks like)

In [16]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']


  prompt = f"""
  Summarize the following conversation.
  {dialogue}
  Summary:
  """

  #Input is constructed prompt instead of dialogue
  inputs = tokenizer(prompt,return_tensors ='pt')
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],max_new_tokens = 50,)[0],
      skip_special_tokens = True

      )

  print(dash_line)
  print('Example',i+1)
  print(dash_line)
  print(f'INPUT PROMPT: \n{prompt}')
  print(dash_line)
  print(f'Human Summary: \n{summary}')
  print(dash_line)
  print(f'Model Summary - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT: 

  Summarize the following conversation.
  #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
  Summary:
  
---------------------------------------------------------------------------------------------------
Human Summary: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Summary - ZERO SHOT:
The train is about to leave.
-

Playing with the prompt - still zero shot (Not giving the summary

In [18]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']


  prompt = f"""
  Summarize the below given dialogue between 2 people.
  {dialogue}
  Summary:
  """

  #Input is constructed prompt instead of dialogue
  inputs = tokenizer(prompt,return_tensors ='pt')
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],max_new_tokens = 50,)[0],
      skip_special_tokens = True

      )

  print(dash_line)
  print('Example',i+1)
  print(dash_line)
  print(f'INPUT PROMPT: \n{prompt}')
  print(dash_line)
  print(f'Human Summary: \n{summary}')
  print(dash_line)
  print(f'Model Summary - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT: 

  Summarize the below given dialogue between 2 people.
  #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
  Summary:
  
---------------------------------------------------------------------------------------------------
Human Summary: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Summary - ZERO SHOT:
The train is ab

THAT's HOW a prompt can change the output

# ----PROMPT ENGINEERING WITH ONE SHOT AND FEW SHOT INFERENCE(PREDICTIONS)----

lets create a function that takes a list of example indicess full, generates a prompt with full examples, then at the end appends the prompt which you want the model to complete(example index to summarize).

In [37]:
def make_prompt(examples_indices_full,example_index_to_summarize):
    prompt = ''
    for index in examples_indices_full:
      dialogue = dataset['test'][index]['dialogue']
      summary = dataset['test'][index]['summary']
      #just keep in mind the stop sequence '{summary}\n\n\n' is important for FLAN-T5 (it tells model to stop going through loop and text generation is complete)
      prompt += f"""
      Dialogue:
      {dialogue}
      What was going on?
      {summary}
      """
    dialogue = dataset['test'][example_index_to_summarize]['dialogue']
    prompt += f"""
    Dialogue:
    {dialogue}
    What was going on?
    """
    return prompt

Construct the prompt for ***one - shot inference*** (means give only one complete example of summary by Human)

In [38]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)

print(one_shot_prompt)


      Dialogue:
      #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
      What was going on?
      #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
      
    Dialogue:
    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processo

passing the prompt for one-shot inference

In [39]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt,return_tensors = 'pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens = True
)

print(dash_line)
print(f'Human Summary:\n{summary}\n')
print(dash_line)
print(f'Model Summary - ONE SHOT LEARNING: \n{output}\n')

---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
Model Summary - ONE SHOT LEARNING: 
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.





---
# **FEW SHOT LEARNING - INFERENCE**


In [40]:
example_indices_full = [40,60,80]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)
print(few_shot_prompt)


      Dialogue:
      #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
      What was going on?
      #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
      
      Dialogue:
      #Person1#: Hey, Frank. I heard you got a new job.
#Person2#: Yeah, Judy. I will be working for the Post Office. It's not a bad job.
#Person1#: Is it true that you have a heavy work schedule?
#Person2#: That's right. I am supposed to work at 5am everyday, and I only get 45 minutes for lunch.
#Person1#: So, why did you apply for such a demanding job?
#Person2#: Well, the government offers its employees excellent health insurance benefits.
#Person1#: Oh, I see. And can y

In [41]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt,return_tensors = 'pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens = True
)

print(dash_line)
print(f'Human Summary:\n{summary}\n')
print(dash_line)
print(f'Model Summary - FEW SHOT INFERENCE: \n{output}\n')

Token indices sequence length is longer than the specified maximum sequence length for this model (877 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
Model Summary - FEW SHOT INFERENCE: 
#Person1 recommends upgrading the system and hardware.



# **GENERATIVE CONFIGURATION Parameters for Inference**

configuration paramets : like max topken, sampling, temperature and top k and top -p token are few examples are config params

Always use the GenerativeConfig Class for configuration parameters

In [42]:
generation_config = GenerationConfig(max_new_tokens = 30,do_sample=True,temperature = 1.2)

summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt,return_tensors = 'pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config = generation_config,
    )[0],
    skip_special_tokens = True
)

print(dash_line)
print(f'Human Summary:\n{summary}\n')
print(dash_line)
print(f'Model Summary - FEW SHOT INFERENCE (with various parameters) : parameters : {generation_config}\n - Output : {output}\n')

---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
Model Summary - FEW SHOT INFERENCE (with various parameters) : parameters : GenerationConfig {
  "do_sample": true,
  "max_new_tokens": 30,
  "temperature": 1.2,
  "transformers_version": "4.27.2"
}

 - Output : #Nocline #Candidate_Four # Candidate: #Scott_Irwin, #Dick_Lan



Experimenting

In [43]:
generation_config = GenerationConfig(max_new_tokens = 50,do_sample=True,temperature = 0.5)

summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt,return_tensors = 'pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config = generation_config,
    )[0],
    skip_special_tokens = True
)

print(dash_line)
print(f'Human Summary:\n{summary}\n')
print(dash_line)
print(f'Model Summary - FEW SHOT INFERENCE (with various parameters) : parameters : {generation_config}\n - Output : {output}\n')

---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
Model Summary - FEW SHOT INFERENCE (with various parameters) : parameters : GenerationConfig {
  "do_sample": true,
  "max_new_tokens": 50,
  "temperature": 0.5,
  "transformers_version": "4.27.2"
}

 - Output : #Person1 wants to upgrade his software and hardware.



In [46]:
generation_config = GenerationConfig(max_new_tokens = 60,do_sample=True,top_k=10,temperature = 0.7)

summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt,return_tensors = 'pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config = generation_config,
    )[0],
    skip_special_tokens = True
)

print(dash_line)
print(f'Human Summary:\n{summary}\n')
print(dash_line)
print(f'Model Summary - FEW SHOT INFERENCE (with various parameters) : parameters : {generation_config}\n - Output : {output}\n')

---------------------------------------------------------------------------------------------------
Human Summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
Model Summary - FEW SHOT INFERENCE (with various parameters) : parameters : GenerationConfig {
  "do_sample": true,
  "max_new_tokens": 60,
  "temperature": 0.7,
  "top_k": 10,
  "transformers_version": "4.27.2"
}

 - Output : #Person1 considers upgrading his system to make it more powerful.

